In [1]:
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
from data_processing import DataProcessor
from IPython.display import display
import pandas as pd

In [2]:
creds_path = '/Users/itsanya/AiCore/MRDC/db_creds.yaml'
b_store_api_key = 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'

db_connector_instance = DatabaseConnector(creds_path) 
data_extractor = DataExtractor(db_connector_instance) 

data_cleaner = DataCleaning()
data_processor = DataProcessor()

In [3]:
#Legacy Users
#Extract
#table_name = 'legacy_users'
#users_df = data_extractor.read_rds_table(table_name)

#View
#display(users_df)

#Clean
#fully_cleaned_users_df = data_cleaner.clean_users_df(users_df)

#View Cleaned DataFrame
#display(fully_cleaned_users_df)

#Upload to dim_users
#db_connector_instance.upload_to_db(df=fully_cleaned_users_df, table_name='dim_users')

In [4]:
#Card Data
#Extract
pdf_link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
card_df = data_extractor.retrieve_pdf_data(pdf_link)

#View
#display(card_df)
#card_df.info()

#Clean
fully_cleaned_card_df = data_cleaner.clean_card_data(card_df)

#View Cleaned DataFrame
display(fully_cleaned_card_df)

#Upload to dim_card_details
db_connector_instance.upload_to_db(df=fully_cleaned_card_df, table_name='dim_card_details')

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'
/Users/itsanya/AiCore/MRDC/data_processing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=column_names, inplace=True)
/Users/itsanya/AiCore/MRDC/data_processing.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[non_conforming_mask, column_name] = pd.NaT


,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09-26,Diners Club / Carte Blanche,2015-11-25 00:00:00
1,349624180933183,10-23,American Express,2001-06-18 00:00:00
2,3529023891650490,06-23,JCB 16 digit,2000-12-26 00:00:00
3,213142929492281,09-27,JCB 15 digit,2011-02-12 00:00:00
4,502067329974,10-25,Maestro,1997-03-13 00:00:00
...,...,...,...,...
15304,180036921556789,12-28,JCB 15 digit,1997-06-06 00:00:00
15305,180018030448512,11-24,JCB 15 digit,2004-06-16 00:00:00
15306,3569953313547220,04-24,JCB 16 digit,2020-02-05 00:00:00
15307,4444521712606810,06-27,VISA 16 digit,2008-06-16 00:00:00


Data uploaded to 'dim_card_details' table successfully.


In [5]:
#Business Store Data
#Extract

#number of stores: - WORKS!
#number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
#data_extractor.set_api_key(b_store_api_key)
#number_of_stores = data_extractor.list_number_of_stores(number_of_stores_endpoint)

#stores data: - DOES NOT WORK...
#store_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}'
#data_extractor.set_api_key(b_store_api_key)
#b_store_df = data_extractor.retrieve_stores_data(store_endpoint)

#View
#display(b_store_df)

#Clean
#fully_cleaned_b_store_df = data_cleaner.clean_store_data(b_store_df)

#View Cleaned DataFrame
#display(fully_cleaned_b_store_df)

#Upload to dim_store_details
#db_connector_instance.upload_to_db(df=fully_cleaned_b_store_df, table_name='dim_store_details')